
# <font color='056938'> **Librerias** </font>

In [ ]:
import os
import sys
import numpy as np
import pandas as pd
import sqlite3 as sql
import plotly.graph_objs as go ### para gráficos
import plotly.express as px
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from ipywidgets import interact ## para análisis interactivo
from sklearn import neighbors ### basado en contenido un solo producto consumido
import joblib
from sklearn.neighbors import NearestNeighbors



# <font color='056938'> **Conexion con Drive** </font>

In [ ]:
# Conexion con Drive
from google.colab import drive
drive.mount('/content/drive')

# Cambio de directorio de Trabajo

path = '/content/drive/MyDrive/Mod2/ANALITICA3'
os.chdir(path)
sys.path.append(path)


Mounted at /content/drive


# <font color='056938'> **Cargar base de datos** </font>

In [ ]:
conn=sql.connect('/content/drive/MyDrive/Mod2/ANALITICA3/data/db_movies')
cur=conn.cursor() ###para funciones que ejecutan sql en base de datos

In [ ]:
### Verificar las tablas que hay disponibles
cur.execute("SELECT name FROM sqlite_master where type='table' ")
cur.fetchall()

[('ratings',),
 ('movies',),
 ('usuarios_sel',),
 ('peliculas_sel',),
 ('ratings_final',),
 ('movies_final',),
 ('full_ratings',)]

In [ ]:
# Cargas la tabla que ya contiene los datos procesados
tabla_completa = pd.read_sql_query("SELECT * FROM full_ratings", conn)

# <font color='056938'> **Tabla Completa con datos procesados** </font>

In [ ]:
tabla_completa

,user_id,movie_id,movie_title,movie_genres,movie_rating,movie_timestamp,movie_year
0,1,1,Toy Story,Adventure|Animation|Children|Comedy|Fantasy,4.0,964982703,1995
1,1,3,Grumpier Old Men,Comedy|Romance,4.0,964981247,1995
2,1,6,Heat,Action|Crime|Thriller,4.0,964982224,1995
3,1,47,Seven (a.k.a. Se7en),Mystery|Thriller,5.0,964983815,1995
4,1,50,"Usual Suspects, The",Crime|Mystery|Thriller,5.0,964982931,1995
...,...,...,...,...,...,...,...
25973,607,3785,Scary Movie,Comedy|Horror,2.0,964743188,2000
25974,607,3793,X-Men,Action|Adventure|Sci-Fi,3.0,964743155,2000
25975,607,3994,Unbreakable,Drama|Sci-Fi,3.0,997847202,2000
25976,607,3996,"Crouching Tiger, Hidden Dragon (Wo hu cang long)",Action|Drama|Romance,2.0,997847173,2000



**Recomendaciones basado en popularidad**

Se construye una tabla que mide la popularidad de las películas basada en las calificaciones otorgadas por los usuarios. Se filtran las películas que tienen un número de calificaciones comprendido entre 50 y 500, ya que este rango garantiza una cantidad suficiente de opiniones

In [ ]:
# Crear tabla de popularidad de películas
movie_popularity = pd.read_sql('''
    SELECT
        movie_title,
        COUNT(*) AS num_ratings,
        ROUND(AVG(movie_rating), 2) AS avg_rating
    FROM full_ratings
    GROUP BY movie_title
    HAVING num_ratings BETWEEN 50 AND 500
''', conn)

In [ ]:
# Ordenar por promedio de calificación y número de votos
popular_movies = movie_popularity.sort_values(['avg_rating', 'num_ratings'], ascending=False)

# Mostrar top 10 películas recomendadas por popularidad
popular_movies.head(10)

,movie_title,num_ratings,avg_rating
163,"Shawshank Redemption, The",195,4.44
10,American History X,77,4.30
148,"Princess Bride, The",96,4.28
87,Goodfellas,85,4.28
133,Monty Python and the Holy Grail,94,4.27
83,"Godfather, The",133,4.25
160,Schindler's List,138,4.24
143,One Flew Over the Cuckoo's Nest,82,4.24
13,Apocalypse Now,77,4.24
63,Dr. Strangelove or: How I Learned to Stop Worr...,69,4.24


En la tabla del top 10 de peliculas recomendadas se evidencian peliculas con numeros considerables de calificaciones y rating superiores a 4.24

 **Fórmula del score ponderado**

Se implementa un  basado en un score ponderado, que combina tanto la calificación promedio como la cantidad de calificaciones que ha recibido cada película. Se aplica una fórmula que da más peso al promedio de calificaciones cuando una película tiene muchas valoraciones y más peso al promedio global cuando tiene pocas.

In [ ]:
# Promedio global de todas las calificaciones
C = pd.read_sql('SELECT AVG(movie_rating) FROM full_ratings', conn).iloc[0, 0]

movie_popularity2 = pd.read_sql('''
    SELECT
        movie_title AS title,
        movie_id,
        COUNT(*) AS num_ratings,
        ROUND(AVG(movie_rating), 2) AS avg_rating
    FROM full_ratings
    GROUP BY movie_id, movie_title
''', conn)


In [ ]:
# Promedio global de todas las calificaciones
C = pd.read_sql('SELECT AVG(movie_rating) FROM full_ratings', conn).iloc[0, 0]

# Percentil 75 de la cantidad de votos
m = movie_popularity2['num_ratings'].quantile(0.75)

# Calcular el score ponderado
movie_popularity2['weighted_score'] = (
    (movie_popularity2['num_ratings'] / (movie_popularity2['num_ratings'] + m)) * movie_popularity2['avg_rating'] +
    (m / (movie_popularity2['num_ratings'] + m)) * C
)


In [ ]:
movie_popularity2 = pd.read_sql('''
    SELECT
        movie_title AS title,
        movie_id,
        COUNT(*) AS num_ratings,
        ROUND(AVG(movie_rating), 2) AS avg_rating
    FROM full_ratings
    GROUP BY movie_id, movie_title
''', conn)


In [ ]:
# Percentil 75 de la cantidad de votos
m = movie_popularity2['num_ratings'].quantile(0.75)

In [ ]:
# Calcular el score ponderado
movie_popularity2['weighted_score'] = (
    (movie_popularity2['num_ratings'] / (movie_popularity2['num_ratings'] + m)) * movie_popularity2['avg_rating'] +
    (m / (movie_popularity2['num_ratings'] + m)) * C
)


In [ ]:
# Ordenar por el score ponderado
movie_popularity_sorted = movie_popularity2.sort_values('weighted_score', ascending=False)

# Mostrar las top 10 películas recomendadas
movie_popularity_sorted[['title', 'weighted_score', 'num_ratings', 'avg_rating']].head(10)

,title,weighted_score,num_ratings,avg_rating
53,"Shawshank Redemption, The",4.260847,195,4.44
118,"Godfather, The",4.081648,133,4.25
87,Schindler's List,4.079060,138,4.24
18,"Usual Suspects, The",4.069336,134,4.23
143,"Princess Bride, The",4.061473,96,4.28
277,Fight Club,4.057984,146,4.20
49,Pulp Fiction,4.057892,203,4.16
44,Star Wars: Episode IV - A New Hope,4.056994,168,4.18
62,Forrest Gump,4.055312,217,4.15
139,Monty Python and the Holy Grail,4.053076,94,4.27


Realizando un comparativo entre las dos tablas de top10 por ranking, se observa que "Shawshank Redemption" se mantiene como la película mejor valorada en ambos métodos, lo que refuerza su popularidad y calidad general. Esto indica que tiene tanto una calificación alta como un número considerable de votaciones. Algunas películas como "American History X" y "Goodfellas", que estaban muy arriba en la primera tabla por promedio simple, ya no aparecen en el top 10 del ranking ponderado. Esto sugiere que aunque tienen calificaciones altas, su número de votos es relativamente bajo.


**Recomendaciones basado en popularidad y el año de publicacion**

Se reliza una consulta que tiene como propósito identificar las películas mejor valoradas por año de publicación, para esto se modifico las tablas iniciales donde realizamos la separacion del año de la pelicula de la columna "movie_title" y se registran en una nueva columna llamada "movie_year".

In [ ]:
# Consulta para obtener las mejores calificaciones por año de publicación
best_rated_by_year = pd.read_sql("""
    SELECT
        movie_year,
        movie_title,
        AVG(CASE WHEN movie_rating = 0 THEN NULL ELSE movie_rating END) AS avg_rat,
        COUNT(CASE WHEN movie_rating = 0 THEN NULL ELSE movie_rating END) AS rat_numb,
        COUNT(*) AS read_num
    FROM full_ratings
    WHERE movie_rating <> 0
    GROUP BY movie_year, movie_title
    ORDER BY movie_year DESC, avg_rat DESC
    LIMIT 10
""", conn)

# Mostrar los resultados
best_rated_by_year


,movie_year,movie_title,avg_rat,rat_numb,read_num
0,2016,Deadpool,3.661290,31,31
1,2014,Guardians of the Galaxy,3.930556,36,36
2,2014,The Imitation Game,3.924242,33,33
3,2014,Interstellar,3.875000,44,44
4,2014,"Grand Budapest Hotel, The",3.714286,35,35
5,2013,"Wolf of Wall Street, The",3.828571,35,35
6,2012,Django Unchained,4.022727,44,44
7,2012,"Dark Knight Rises, The",3.931373,51,51
8,2012,Skyfall,3.878378,37,37
9,2012,"Avengers, The",3.788889,45,45


**Recomendaciones basado en popularidad y por género**

Esta consulta permite identificar las películas más populares dentro de cada género, basándose en el promedio de calificaciones recibidas por los usuarios

In [ ]:
query = '''
WITH exploded_genres AS (
    SELECT
        movie_title,
        movie_rating,
        TRIM(je.value) AS genre
    FROM full_ratings,
         json_each('["' || REPLACE(movie_genres, '|', '","') || '"]') AS je
)
SELECT
    genre,
    movie_title,
    COUNT(*) AS num_ratings,
    ROUND(AVG(movie_rating), 2) AS avg_rating
FROM exploded_genres
WHERE movie_rating <> 0
GROUP BY genre, movie_title
HAVING num_ratings >= 10
ORDER BY genre, avg_rating DESC
'''

genre_popularity = pd.read_sql(query, conn)

# Mostrar las top 5 películas por cada género
genre_popularity.groupby('genre').head(5)


,genre,movie_title,num_ratings,avg_rating
0,Action,North by Northwest,38,4.29
1,Action,"Princess Bride, The",96,4.28
2,Action,City of God (Cidade de Deus),45,4.27
3,Action,Apocalypse Now,77,4.24
4,Action,Fight Club,146,4.20
...,...,...,...,...
1311,Western,"Good, the Bad and the Ugly, The (Buono, il bru...",37,4.19
1312,Western,Butch Cassidy and the Sundance Kid,37,4.07
1313,Western,Blazing Saddles,32,4.06
1314,Western,Django Unchained,44,4.02


# <font color='056938'> **Sistema de recomendaciones interectivo** </font>

 Implementamos un sistema de recomendaciones de películas interactivo utilizando ipywidgets, permitiendo al usuario explorar las películas mejor valoradas según distintos criterios como el genero, el año de publicacion y por su popularidad.

In [ ]:
from ipywidgets import interact, Dropdown, IntSlider

# Lista de géneros más opción "Todos"
generos = ['Todos'] + sorted(genre_popularity['genre'].unique().tolist())

def recomendaciones_interactivas(tipo, min_ratings=50, year=2000, genre='Todos'):
    if tipo == 'Popularidad':
        df = movie_popularity[movie_popularity['num_ratings'] >= min_ratings]
        df = df.sort_values(['avg_rating', 'num_ratings'], ascending=False)
        return df[['movie_title', 'avg_rating', 'num_ratings']].head(10)

    elif tipo == 'Por año':
        df = tabla_completa[(tabla_completa['movie_year'] == year) &
                            (tabla_completa['movie_rating'] >= 0)]
        df_grouped = df.groupby('movie_title').agg(
            avg_rating=('movie_rating', 'mean'),
            num_ratings=('movie_rating', 'count')
        ).reset_index()
        df_filtered = df_grouped[df_grouped['num_ratings'] >= min_ratings]
        df_sorted = df_filtered.sort_values(['avg_rating', 'num_ratings'], ascending=False)
        return df_sorted.head(10)

    elif tipo == 'Por género':
        if genre == 'Todos':
            df = genre_popularity[genre_popularity['num_ratings'] >= min_ratings]
        else:
            df = genre_popularity[(genre_popularity['genre'] == genre) &
                                  (genre_popularity['num_ratings'] >= min_ratings)]
        df = df.sort_values(['avg_rating', 'num_ratings'], ascending=False)
        return df[['movie_title', 'avg_rating', 'num_ratings', 'genre']].head(10)

# # Widgets
# interact(
#     recomendaciones_interactivas,
#     tipo=Dropdown(options=['Popularidad', 'Por año', 'Por género'], description='Tipo:'),
#     min_ratings=IntSlider(min=10, max=500, step=10, value=50, description='Mín. ratings'),
#     year=IntSlider(min=1990, max=2020, step=1, value=2000, description='Año:'),
#     genre=Dropdown(options=generos, description='Género:')
# )


interactive(children=(Dropdown(description='Tipo:', options=('Popularidad', 'Por año', 'Por género'), value='P…

<function __main__.recomendaciones_interactivas(tipo, min_ratings=50, year=2000, genre='Todos')>